# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [ ]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import google.generativeai as genai
import re

In [36]:
# Initialize and constants

load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')
api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("AIzaSy"):
    print("An API key was found, but it doesn't start with AIzaSy; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


# MODEL = 'gpt-4o-mini'
# openai = OpenAI()
model = genai.GenerativeModel("gemini-2.5-flash")
genai.configure()
print(type(model))

API key found and looks good so far!
<class 'google.generativeai.generative_models.GenerativeModel'>


In [37]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [38]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [39]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""



In [40]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [41]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [42]:
print(get_links_user_prompt(ed))


Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edward

In [43]:
def parse_gemini_response(response):
    # Access candidates directly on response
    text = response.candidates[0].content.parts[0].text
    
    # Remove markdown JSON code block wrappers
    json_str = re.sub(r"```json\s*|\s*```", "", text).strip()
    
    # Parse and return JSON
    return json.loads(json_str)

In [44]:
def get_links(url):
    website = Website(url)
    # response = openai.chat.completions.create(
    #     model=MODEL,
    #     messages=[
    #         {"role": "system", "content": link_system_prompt},
    #         {"role": "user", "content": get_links_user_prompt(website)}
    #   ],
    #     response_format={"type": "json_object"}
    # )
    # result = response.choices[0].message.content
    # return json.loads(result)

    messages =[
        {
            "role": "user",
            "parts": [f"{link_system_prompt}\n\n{get_links_user_prompt(website)}"]
        }
    ]
    # response_format={"type": "json_object"}

    response = model.generate_content(messages)
    # print(response)

    
    return parse_gemini_response(response)
    

In [45]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/baidu/ERNIE-4.5-21B-A3B-Thinking',
 '/tencent/SRPO',
 '/tencent/HunyuanImage-2.1',
 '/Qwen/Qwen3-Next-80B-A3B-Instruct',
 '/google/embeddinggemma-300m',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/multimodalart/wan-2-2-first-last-frame',
 '/spaces/Qwen/Qwen3-ASR-Demo',
 '/spaces/tencent/HunyuanImage-2.1',
 '/spaces',
 '/datasets/HuggingFaceFW/finepdfs',
 '/datasets/HuggingFaceM4/FineVision',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/LucasFang/FLUX-Reason-6M',
 '/datasets/jupyter-agent/jupyter-agent-dataset',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 

In [49]:
get_links("https://huggingface.co")

{'links': [{'type': 'careers page',
   'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'enterprise solutions page',
   'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'endpoints page', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'company info page', 'url': 'https://huggingface.co/brand'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [50]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [55]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [56]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [57]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'learning page', 'url': 'https://huggingface.co/learn'}, {'type': 'company profile page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/SRPO\nUpdated\nabout 9 hours ago\n•\n3.08k\n•\n690\nbaidu/ERNIE-4.5-21B-A3B-Thinking\nUpdated\n4 days ago\n•\n102k\n•\n689\ntencent/HunyuanImage-2.1\nUpdated\n3 days ago\n•\n5.53k\n•\n592\nQwen/Qwen3-Next-80B-A3B-Instruct\nUpdated\nabout 7 hours ago\n•\n198k\n•\n509\ngoogle/embeddinggemma-300m\nUpdated\n5 days ago\n•\n163k\n•\n783\nBrowse 1M+ models\nSpaces\nRunning\n13.4k\n13.4k\n

In [68]:
def create_brochure(company_name, url):
    
    messages =[
        {
            "role": "user",
            "parts": [f"{system_prompt}\n\n{get_brochure_user_prompt(company_name, url)}"]
        }
    ]
    
    # for Open AI
    # response = openai.chat.completions.create(
    #     model=MODEL,
    #     messages=[
    #         {"role": "system", "content": system_prompt},
    #         {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
    #       ],
    # )
    # result = response.choices[0].message.content
    result = model.generate_content(messages)

    result_text = result.candidates[0].content.parts[0].text
    display(Markdown(result_text))

In [69]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise solutions', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'learning resources', 'url': 'https://huggingface.co/learn'}]}


# Hugging Face: The AI Community Building the Future

Hugging Face is the leading platform where the global machine learning (ML) community collaborates on models, datasets, and applications, accelerating the future of artificial intelligence. We empower individuals and organizations to create, discover, and collaborate on ML better, fostering an open and innovative ecosystem.

---

### For Prospective Customers: Elevate Your AI Development

Hugging Face offers an unparalleled suite of tools and resources designed to streamline and accelerate your machine learning workflows:

*   **Vast Resources:** Access over 1 million models, 250,000+ datasets, and 400,000+ applications (Spaces) across all modalities—text, image, video, audio, and even 3D.
*   **Seamless Collaboration:** Our platform provides a collaborative environment to host and work on unlimited public models, datasets, and applications, enabling your team to move faster.
*   **Powerful Compute & Enterprise Solutions:**
    *   **Compute:** Deploy on optimized Inference Endpoints or upgrade your Spaces applications to GPUs with ease, starting at competitive hourly rates.
    *   **Team & Enterprise:** Benefit from advanced features like enterprise-grade security, access controls, dedicated support, Single Sign-On (SSO), audit logs, resource groups, and private datasets viewer.
*   **Industry-Leading Open Source Tools:** We build the foundation of ML tooling with the community, offering widely adopted libraries such as Transformers, Diffusers, TRL, PEFT, and more.

**Trusted by Over 50,000 Organizations:** Leading innovators like Google, Microsoft, Amazon, Meta, AI2, Intel, Grammarly, and Writer leverage Hugging Face to power their AI initiatives.

---

### For Investors: Investing in the Foundation of AI

Hugging Face represents a strategic investment in the core infrastructure of modern AI. We are at the forefront of the machine learning revolution, driven by:

*   **Market Leadership:** We are the definitive collaboration platform and a central hub for the global ML community, with unparalleled user engagement and content growth.
*   **Robust Ecosystem:** Our platform supports a comprehensive lifecycle of ML development, from research and data preparation to model deployment and application sharing.
*   **Strong Commercial Offerings:** Our paid Compute and Enterprise solutions provide scalable revenue streams by addressing critical needs for performance, security, and team management for large organizations.
*   **Open Source Advantage:** Our leadership in open source ML tooling (e.g., Transformers, Diffusers) establishes a powerful network effect, driving adoption and reinforcing our platform's central role.
*   **Massive Adoption:** With over 50,000 organizations and a vibrant community of millions, Hugging Face is foundational to how AI is being built today and in the future.

---

### For Recruits: Join the Movement to Build the Future of AI

Are you passionate about machine learning and its potential to shape the future? At Hugging Face, you'll be part of a vibrant, innovative, and impactful environment:

*   **Our Mission:** We are "the AI community building the future." Join us in creating, discovering, and collaborating on ML better, contributing to the foundational tools that empower millions.
*   **Collaborative Culture:** We embody a strong community spirit, valuing open source contributions, shared knowledge, and collective innovation. Our work is deeply rooted in fostering collaboration within the ML world.
*   **Impactful Work:** Your contributions will directly influence the development of state-of-the-art AI, working on projects that are widely adopted and make a tangible difference in the field.
*   **Growth and Learning:** Surround yourself with brilliant minds and cutting-edge technology, with ample opportunities to learn, grow, and build your ML profile.

**Ready to make your mark in AI?** Explore our career opportunities and become a part of the team at Hugging Face.

---

### Discover Hugging Face Today!

Whether you're looking to accelerate your AI projects, invest in the future of machine learning, or build a career at the cutting edge of technology, Hugging Face is your destination.

**Visit us at huggingface.co to learn more, explore our platform, or browse our open positions.**